# Django Tutorial (part 3)
We should be at the project root:

In [1]:
cd "$HOME/workspace/python-course/Level 4/01 Django"
pwd

/home/chris/workspace/python-course/Level 4/01 Django


In [2]:
cd workspace/mysite
tree -I __pycache__ "$PWD"

/home/chris/workspace/python-course/Level 4/01 Django/workspace/mysite
├── db.sqlite3
├── manage.py
├── mysite
│   ├── asgi.py
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── tennis
    ├── admin.py
    ├── apps.py
    ├── forms.py
    ├── __init__.py
    ├── migrations
    │   ├── 0001_initial.py
    │   └── __init__.py
    ├── models.py
    ├── templates
    │   ├── home.html
    │   └── tennis
    │       └── file3.html
    ├── tests.py
    ├── urls.py
    └── views.py

6 directories, 19 files


Restart the server

In [3]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

[1] 28022


Django has an interactive Python shell which we can use to add data to `sqlite`.  Again, we'll use Unix hereis documents to controll the shell.  Of course, in practice, you would just use the command line.

In [4]:
python manage.py shell << EOF &
from tennis.models import Match  # Import the model classes we just wrote
print(Match.objects.all())
EOF

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
December 03, 2024 - 13:10:30
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

python manage.py shell << EOF &^J
[2] 28086


We can now create data for a new Match:

In [5]:
python manage.py shell << EOF
from tennis.models import Match # Import the model classes we just wrote.

def createMatch(home, score1, away, score2):
    m = Match()
    m.play(home, score1, away, score2)
    # Save the objects into the database. You have to call save() explicitly.
    m.save()
    
createMatch("Blue", 2, "Green", 7)
createMatch("Blue", 6, "Red", 3)
createMatch("Green", 4, "Purple", 0)
createMatch("Purple", 1, "Blue", 2)

EOF

<QuerySet [<Match: Blue 2-7 Green>, <Match: Blue 6-3 Red>, <Match: Green 4-0 Purple>, <Match: Purple 1-2 Blue>]>
[2]+  Done                    python manage.py shell <<EOF
from tennis.models import Match  # Import the model classes we just wrote
print(Match.objects.all())
EOF



We can add some more matches:

In [6]:
# Access model field values via Python attributes.
python manage.py shell << EOF
from tennis.models import Match # Import the model classes we just wrote.


print(Match.objects.all())
m = Match.objects.all()[0]
m.play("Blue", 5, "Red", 3)
m.play("Blue", 2, "Green", 7)
m.play("Red", 5, "Purple", 5)
m.play("Red", 5, "Purple", 5)

print(m)
EOF

<QuerySet [<Match: Blue 2-7 Green>, <Match: Blue 6-3 Red>, <Match: Green 4-0 Purple>, <Match: Purple 1-2 Blue>, <Match: Blue 2-7 Green>, <Match: Blue 6-3 Red>, <Match: Green 4-0 Purple>, <Match: Purple 1-2 Blue>]>
Red 5-5 Purple


That's a bit cryptic.  We can improve the print out by changing the model slightly.
The current model looks like:

In [7]:
cat tennis/models.py

from django.db import models

class Match(models.Model):
    homeTeam = models.CharField(max_length=20)
    homeScore = models.IntegerField(default=0)
    awayTeam = models.CharField(max_length=20)
    awayScore = models.IntegerField(default=0)

    def play(self, home, score1, away, score2):
        self.homeTeam = home
        self.homeScore = score1
        self.awayTeam = away
        self.awayScore = score2
    def __str__(self):
         return f"{self.homeTeam} {self.homeScore}-{self.awayScore} {self.awayTeam}"


Change the `Match` class by adding `__str__` methods:

In [8]:
cat << EOF > tennis/models.py
from django.db import models

class Match(models.Model):
    homeTeam = models.CharField(max_length=20)
    homeScore = models.IntegerField(default=0)
    awayTeam = models.CharField(max_length=20)
    awayScore = models.IntegerField(default=0)

    def play(self, home, score1, away, score2):
        self.homeTeam = home
        self.homeScore = score1
        self.awayTeam = away
        self.awayScore = score2
    def __str__(self):
         return f"{self.homeTeam} {self.homeScore}-{self.awayScore} {self.awayTeam}"
EOF

python manage.py migrate

/home/chris/workspace/python-course/src/38 Django/workspace/mysite/tennis/models.py changed, reloading.
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions, tennis
Running migrations:
  No migrations to apply.


Now run the queries again (to see text printed):

In [9]:
# Access model field values via Python attributes.
python manage.py shell << EOF
from tennis.models import Match  # Import the model classes we just wrote.

for match in Match.objects.all():
    print(match)
EOF

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
December 03, 2024 - 13:11:09
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

# Access model field values via Python attributes.^J
Blue 2-7 Green
Blue 6-3 Red
Green 4-0 Purple
Purple 1-2 Blue
Blue 2-7 Green
Blue 6-3 Red
Green 4-0 Purple
Purple 1-2 Blue


Other things you can do in the shell:

Find all the results for the "Blue" team:

In [10]:
python manage.py shell << EOF
from tennis.models import Match
print(Match.objects.filter(homeTeam="Blue"))
EOF

python manage.py shell << EOF
from tennis.models import Match
print(Match.objects.filter(homeTeam="Blue"))
EOF
<QuerySet [<Match: Blue 2-7 Green>, <Match: Blue 6-3 Red>]>


: 1

Let's delete one of the mathes using delete():

In [11]:
python manage.py shell << EOF
from tennis.models import Match
m = Match.objects.filter(awayTeam="Green")
m.delete()

for match in Match.objects.all():
    print(match)
EOF

python manage.py shell << EOF
from tennis.models import Match
m = Match.objects.filter(awayTeam="Green")
m.delete()

for match in Match.objects.all():
    print(match)
EOF
Blue 6-3 Red
Green 4-0 Purple
Purple 1-2 Blue


: 1

We can use a webapp to see all of the above.  First create superuser for Django Admin by
running the following command in xterm (name and password of your choice):

In [16]:
xterm -fg black -bg white -fa 'Monospace' -fs 14 -e 'python manage.py createsuperuser' &

[29/Nov/2024 12:45:48] "GET /admin/ HTTP/1.1" 302 0
[29/Nov/2024 12:45:48] "GET /admin/login/?next=/admin/ HTTP/1.1" 200 4160
[29/Nov/2024 12:45:48] "GET /static/admin/js/theme.js HTTP/1.1" 200 1653
[29/Nov/2024 12:45:48] "GET /static/admin/js/nav_sidebar.js HTTP/1.1" 200 3063
[29/Nov/2024 12:45:48] "GET /static/admin/css/dark_mode.css HTTP/1.1" 200 2804
[29/Nov/2024 12:45:48] "GET /static/admin/css/login.css HTTP/1.1" 200 951
[29/Nov/2024 12:45:48] "GET /static/admin/css/nav_sidebar.css HTTP/1.1" 200 2810
[29/Nov/2024 12:45:48] "GET /static/admin/css/responsive.css HTTP/1.1" 200 17972
[29/Nov/2024 12:45:48] "GET /static/admin/css/base.css HTTP/1.1" 200 22092
Not Found: /favicon.ico
[29/Nov/2024 12:45:48] "GET /favicon.ico HTTP/1.1" 404 2358
[29/Nov/2024 12:46:13] "POST /admin/login/?next=/admin/ HTTP/1.1" 200 4327
[29/Nov/2024 12:46:34] "POST /admin/login/?next=/admin/ HTTP/1.1" 200 4327
xterm -fg black -bg white -fa 'Monospace' -fs 14 -e 'python manage.py createsuperuser' &
[3] 17262

: 1

Register the `Match` class with the admin app:

In [17]:
cat << EOF > tennis/admin.py
from django.contrib import admin
from .models import Match
admin.site.register(Match)
EOF

cat << EOF > tennis/admin.py
from django.contrib import admin
from .models import Match
admin.site.register(Match)
EOF
[3]+  Done                    xterm -fg black -bg white -fa 'Monospace' -fs 14 -e 'python manage.py createsuperuser'


: 1

Check the registration worked:

In [18]:
cat tennis/admin.py

/home/chris/workspace/python-course/src/38 Django/workspace/mysite/tennis/admin.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
November 29, 2024 - 12:47:42
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

cat tennis/admin.py^Jcat tennis/admin.py
from django.contrib import admin
from .models import Match
admin.site.register(Match)


: 1

Now fire up the admin app and login as the superuser.  You can now investigate all the `Match` results.

In [19]:
firefox http://127.0.0.1:7000/admin/ &

firefox http://127.0.0.1:7000/admin/ &
[2] 17376


: 1